In [ ]:
!pip install -q mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 64.4 MB/s eta 0:00:00


In [ ]:
import os
import tensorflow as tf
from mediapipe_model_maker import gesture_recognizer
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from PIL import Image
import numpy as np
from PIL import Image

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/Colab Notebooks/garudahacks-ai"

In [ ]:
images_path = os.path.join(base_path, 'asl_dataset')
print(images_path)

/content/drive/MyDrive/Colab Notebooks/garudahacks-ai/asl_dataset


In [ ]:
labels = os.listdir(images_path)


print('Labels : ', labels)
print('Len Labels : ', len(labels))

Labels :  ['4', '9', '6', '0', '8', '3', '1', '5', '2', '7', 'h', 'i', 'd', 'f', 'b', 'a', 'j', 'g', 'c', 'e', 'o', 'l', 'k', 'm', 'n', 'none', 's', 'r', 'q', 'p', 'y', 'u', 'w', 'v', 'z', 't', 'x']
Len Labels :  37


In [ ]:
total_images = 0
for label in labels:
    label_dir = os.path.join(images_path, label)
    file_names = os.listdir(label_dir)

    total_images += len(file_names)

    for index, file_name in enumerate(file_names):
        file_path = os.path.join(label_dir, file_name)
        new_file_path = os.path.join(label_dir, f'{index}.jpg')
        if file_path != new_file_path:
            os.rename(file_path, new_file_path)
        try:
            with Image.open(new_file_path) as img:
                if img.width != 400 or img.height != 400:
                    print('Invalid Image : ', new_file_path)
        except Exception as e:
            print(e)

print('Total Images : ', total_images)

Total Images :  2515


In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=images_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams(),
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
hparams = gesture_recognizer.HParams(export_dir="asl_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 37)                4773      
 out (Dense)                                                     
                                                             

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

141/141 [==============================] - 4s 4ms/step - loss: 0.2626 - categorical_accuracy: 0.8298
Test loss:0.2625753879547119, Test accuracy:0.8297872543334961


In [ ]:
model_name = "model_2.task"
model.export_model(model_name)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite


In [ ]:
from google.colab import files
files.download(f"asl_model/{model_name}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_path = os.path.join(base_path, 'test.jpg')
print(test_path)


/content/drive/MyDrive/Colab Notebooks/garudahacks-ai/test.jpg


In [ ]:
img = cv2.imread(test_path)
print(img)

[[[ 58  54  65]
  [ 52  48  59]
  [ 47  43  54]
  ...
  [227 215 197]
  [223 211 193]
  [223 211 193]]

 [[ 57  53  64]
  [ 53  49  60]
  [ 50  46  57]
  ...
  [227 215 197]
  [221 209 191]
  [221 209 191]]

 [[ 57  53  64]
  [ 56  52  63]
  [ 54  50  61]
  ...
  [226 214 196]
  [226 214 196]
  [226 214 196]]

 ...

 [[ 62 110 138]
  [ 63 111 139]
  [ 64 112 140]
  ...
  [144 173 182]
  [155 184 191]
  [155 184 191]]

 [[ 62 110 138]
  [ 63 111 139]
  [ 64 112 140]
  ...
  [144 173 182]
  [152 181 188]
  [152 181 188]]

 [[ 62 110 138]
  [ 63 111 139]
  [ 64 112 140]
  ...
  [144 173 182]
  [149 178 185]
  [149 178 185]]]


In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = os.path.abspath(f"asl_model/{model_name}")
print('Model Path : ', model_path)

recognizer = vision.GestureRecognizer.create_from_model_path(model_path)

image = mp.Image.create_from_file(test_path)

recognition_result = recognizer.recognize(image)

top_gesture = recognition_result.gestures[0][0]
print('Gesture : ', recognition_result.gestures[0])
print('Top Gesture : ', top_gesture)
print(f"Gesture recognized: {top_gesture.category_name} ({top_gesture.score})")

Model Path :  /content/asl_model/model_2.task
Gesture :  [Category(index=-1, score=0.8560377955436707, display_name='', category_name='')]
Top Gesture :  Category(index=-1, score=0.8560377955436707, display_name='', category_name='')
Gesture recognized:  (0.8560377955436707)


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
!mv asl_model/new_model.task "/content/drive/My Drive/Colab Notebooks/garudahacks-pre-ai/"

mv: cannot stat 'asl_model/new_model.task': No such file or directory
